In [ ]:

install.packages("BinNor", dependencies = TRUE)
install.packages("plyr", dependencies = TRUE)
install.packages("caret", dependencies = TRUE)
install.packages("GMDH2", dependencies = TRUE)
install.packages("caTools", dependencies = TRUE)

library("BinNor")
library("plyr")
library("caret")
library("GMDH2")
library("caTools")


start <- Sys.time()

######### Radiomics features
radiomics_features <- read.delim("~/radiomics_features.txt")
row.names(radiomics_features) <- radiomics_features[,1]
radiomics_features <- radiomics_features[,-1]
dim(radiomics_features)


######### Clinical data
clinical_data <- read.delim("~/NSCLCR01_Clinical_data.txt")
row.names(clinical_data) <- clinical_data[,1]
clinical_data <- clinical_data[,-1]


######### Genetic data
genetic_data_first <- read.delim("~/GSE103584_R01_NSCLC_RNAseq.txt")
genetic_data <- t(genetic_data_first)
genetic_data <- as.data.frame(genetic_data)
colnames(genetic_data) <- genetic_data_first[,1]
genetic_data <- genetic_data[-1,]


######### Removing columns containing NA in genetic data
na_genes <- NULL
for(i in 1:dim(genetic_data)[2])
{
  if(sum(is.na(genetic_data[,i]))>0){
    na_genes <- c(na_genes, names(genetic_data)[i])
  }
}

genetic_data_filtered <- subset(genetic_data, select = ! names(genetic_data) %in% na_genes)
dim(genetic_data_filtered)

row.names(genetic_data_filtered) <- gsub('R01.', 'R01-', row.names(genetic_data_filtered))
genetic_data_filtered <- as.data.frame(genetic_data_filtered)
genetic_data_filtered <- as.matrix(genetic_data_filtered)
storage.mode(genetic_data_filtered) <- "double"


######### Selecting common samples for all data sets
common_samples <- intersect(row.names(radiomics_features), row.names(genetic_data_filtered))
radiomics_features_common <- radiomics_features[rownames(radiomics_features) %in% common_samples, ]  # Extract rows from data
genetic_data_common <- genetic_data_filtered[rownames(genetic_data_filtered) %in% common_samples, ]
clinical_data_common <- clinical_data[rownames(clinical_data) %in% common_samples, ]

summary(as.factor(clinical_data_common$Histology))

nsclc_nos <- row.names(clinical_data_common[clinical_data_common$Histology=="NSCLC NOS (not otherwise specified)",])
radiomics_features_last <- radiomics_features_common[!rownames(radiomics_features_common) %in% nsclc_nos, ] 
genetic_data_last <- genetic_data_common[!rownames(genetic_data_common) %in% nsclc_nos, ] 
clinical_data_last <- clinical_data_common[!rownames(clinical_data_common) %in% nsclc_nos, ] 

data <- cbind(clinical_data_last$Histology, radiomics_features_last, genetic_data_last)
colnames(data)[1] <- "Histology"

dim(data)

data$Histology <- plyr::revalue(data$Histology, c("Adenocarcinoma" = "1", 
                                                  "Squamous cell carcinoma" = "0"))
data$Histology <- as.numeric(data$Histology)

data[,34:5301] <- log2(data[,34:5301])
data[,2:33] <- scale(data[,2:33])


########################################################################
############################ Data simulation############################
########################################################################

M <- 100

sim_data <- NULL
results <- NULL


for(i in 1:M){
  
  set.seed(i)
  
  no.rows=58
  no.bin=1 
  prop.vec.bin=42/58 #########################
  no.nor=5300
  cmat = cor(data[,1:5301])
  data_numeric <- data[,2:5301]
  mean.vec.nor=lapply(data_numeric, mean)
  var.nor=lapply(data_numeric, var)
  
  near.m <- nearPD(cmat)
  diag(near.m$mat) <- 1
  
  sigma.star=compute.sigma.star(no.bin=1, no.nor=5300, prop.vec.bin=prop.vec.bin, 
                                 corr.mat= as.matrix(near.m$mat))
  
  sim_data=jointly.generate.binary.normal(no.rows,no.bin,no.nor,prop.vec.bin,
                                         as.numeric(mean.vec.nor),
                                         as.numeric(var.nor), 
                                         sigma_star=sigma.star$sigma_star, 
                                        continue.with.warning=TRUE)
  sim_data <- as.data.frame(sim_data)
  names(sim_data) <- names(data)[1:5301]
  
  ### Train-test sets
  index=sample.split(sim_data $Histology, SplitRatio=0.7)
  train=sim_data[index,]
  test=sim_data[!index,]
  summary(as.factor(train$Histology))
  summary(as.factor(test$Histology))
  
  train$Histology <- as.factor(train$Histology)
  train$Histology <- droplevels(train$Histology)
  summary(train$Histology)
  class(train$Histology)
  levels(train$Histology) <- gsub(" ", "_", levels(train$Histology))
  levels(train$Histology)
  
  test$Histology <- as.factor(test$Histology)
  test$Histology <- droplevels(test$Histology)
  summary(test$Histology)
  class(test$Histology)
  levels(test$Histology) <- gsub(" ", "_", levels(test$Histology))
  levels(test$Histology)
  
  test$Histology <- plyr::revalue(test$Histology, 
                                  c("1" = "Adenocarcinoma", 
                                    "0" = "Squamous_cell_carcinoma"))
  
  ####################################################################################
  ######################################## AUC #######################################
  ####################################################################################
  
  AUC_results <- colAUC(train[,2:33], train$Histology)
  View(AUC_results)
  AUC_names <- store <- NULL
  for(j in 1:length(AUC_results)){
    if(AUC_results[1,][j]>0.60){
      AUC_names <- names(AUC_results[1,][j])
      store <- cbind(store, AUC_names)
    }
  }
  length(store)
  radiomics_data_AUC <- subset(train[,2:33], select = store[1,])
  radiomics_data_AUC_Histology <- cbind(train$Histology, radiomics_data_AUC)
  names(radiomics_data_AUC_Histology)[1] <- "Histology"
  
  
  AUC_results <- colAUC(train[,34:5301], train$Histology)
  View(AUC_results)
  AUC_names <- store <- NULL
  for(j in 1:length(AUC_results)){
    if(AUC_results[1,][j]>0.60){
      AUC_names <- names(AUC_results[1,][j])
      store <- cbind(store, AUC_names)
    }
  }
  length(store)
  
  genetic_data_AUC <- subset(train[,34:5301], select = store[1,])
  dim(genetic_data_AUC)
  genetic_data_AUC_histology <- cbind(train$Histology, genetic_data_AUC)
  names(genetic_data_AUC_histology)[1] <- "Histology"
  
  
  all_AUC <- cbind(radiomics_data_AUC_Histology, genetic_data_AUC_histology[,-1])
  
  radiomics_data_AUC_Histology$Histology <- plyr::revalue(radiomics_data_AUC_Histology$Histology, 
                                                          c("1" = "Adenocarcinoma", 
                                                            "0" = "Squamous_cell_carcinoma"))
  
  genetic_data_AUC_histology$Histology <- plyr::revalue(genetic_data_AUC_histology$Histology, 
                                                        c("1" = "Adenocarcinoma", 
                                                          "0" = "Squamous_cell_carcinoma"))
  
  all_AUC$Histology <- plyr::revalue(all_AUC$Histology, 
                                     c("1" = "Adenocarcinoma", 
                                       "0" = "Squamous_cell_carcinoma"))
  
  
  #Cross-validation
  ctrl = trainControl(method="cv", number=5, returnResamp = "final", summaryFunction = prSummary, classProbs = TRUE,
                      savePredictions = T, verbose=F)
  
  ########## Elastic Net
  
  model_en_AUC_radiomics = train(Histology~., data = radiomics_data_AUC_Histology, method = "glmnet", trControl = ctrl, tuneLength=5)
  pred_en_AUC_radiomics <- predict(model_en_AUC_radiomics, test[,-1])
  perf_en_AUC_radiomics <- confMat(pred_en_AUC_radiomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_en_AUC_genomics = train(Histology~., data = genetic_data_AUC_histology, method = "glmnet", trControl = ctrl, tuneLength=5)
  pred_en_AUC_genomics <- predict(model_en_AUC_genomics, test[,-1])
  perf_en_AUC_genomics <- confMat(pred_en_AUC_genomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_en_AUC_all = train(Histology~., data = all_AUC, method = "glmnet", trControl = ctrl, tuneLength=5)
  pred_en_AUC_all <- predict(model_en_AUC_all, test[,-1])
  perf_en_AUC_all <- confMat(pred_en_AUC_all, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  
  ########## Random Forest
  model_rf_AUC_radiomics = train(Histology~., data = radiomics_data_AUC_Histology, method = "rf", trControl = ctrl, tuneLength=5)
  pred_rf_AUC_radiomics <- predict(model_rf_AUC_radiomics, test[,-1])
  perf_rf_AUC_radiomics <- confMat(pred_rf_AUC_radiomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_rf_AUC_genomics = train(Histology~., data = genetic_data_AUC_histology, method = "rf", trControl = ctrl, tuneLength=5)
  pred_rf_AUC_genomics <- predict(model_rf_AUC_genomics, test[,-1])
  perf_rf_AUC_genomics <- confMat(pred_rf_AUC_genomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_rf_AUC_all = train(Histology~., data = all_AUC, method = "rf", trControl = ctrl, tuneLength=5)
  pred_rf_AUC_all <- predict(model_rf_AUC_all, test[,-1])
  perf_rf_AUC_all <- confMat(pred_rf_AUC_all, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all

  
  ########## Support Vector Machines - Linear
  model_svm_linear_AUC_radiomics = train(Histology~., data = radiomics_data_AUC_Histology, method = "svmLinear", trControl = ctrl, tuneLength=5)
  pred_svm_linear_AUC_radiomics <- predict(model_svm_linear_AUC_radiomics, test[,-1])
  perf_svm_linear_AUC_radiomics <- confMat(pred_svm_linear_AUC_radiomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_svm_linear_AUC_genomics = train(Histology~., data = genetic_data_AUC_histology, method = "svmLinear", trControl = ctrl, tuneLength=5)
  pred_svm_linear_AUC_genomics <- predict(model_svm_linear_AUC_genomics, test[,-1])
  perf_svm_linear_AUC_genomics <- confMat(pred_svm_linear_AUC_genomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_svm_linear_AUC_all = train(Histology~., data = all_AUC, method = "svmLinear", trControl = ctrl, tuneLength=5)
  pred_svm_linear_AUC_all <- predict(model_svm_linear_AUC_all, test[,-1])
  perf_svm_linear_AUC_all <- confMat(pred_svm_linear_AUC_all, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all

  
  ########## Support Vector Machines - Radial
  model_svm_radial_AUC_radiomics = train(Histology~., data = radiomics_data_AUC_Histology, method = "svmRadial", trControl = ctrl, tuneLength=5)
  pred_svm_radial_AUC_radiomics <- predict(model_svm_radial_AUC_radiomics, test[,-1])
  perf_svm_radial_AUC_radiomics <- confMat(pred_svm_radial_AUC_radiomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_svm_radial_AUC_genomics = train(Histology~., data = genetic_data_AUC_histology, method = "svmRadial", trControl = ctrl, tuneLength=5)
  pred_svm_radial_AUC_genomics <- predict(model_svm_radial_AUC_genomics, test[,-1])
  perf_svm_radial_AUC_genomics <- confMat(pred_svm_radial_AUC_genomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_svm_radial_AUC_all = train(Histology~., data = all_AUC, method = "svmRadial", trControl = ctrl, tuneLength=5)
  pred_svm_radial_AUC_all <- predict(model_svm_radial_AUC_all, test[,-1])
  perf_svm_radial_AUC_all <- confMat(pred_svm_radial_AUC_all, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  
  ########## Support Vector Machines - Polynomial
  model_svm_poly_AUC_radiomics = train(Histology~., data = radiomics_data_AUC_Histology, method = "svmPoly", trControl = ctrl, tuneLength=5)
  pred_svm_poly_AUC_radiomics <- predict(model_svm_poly_AUC_radiomics, test[,-1])
  perf_svm_poly_AUC_radiomics <- confMat(pred_svm_poly_AUC_radiomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_svm_poly_AUC_genomics = train(Histology~., data = genetic_data_AUC_histology, method = "svmPoly", trControl = ctrl, tuneLength=5)
  pred_svm_poly_AUC_genomics <- predict(model_svm_poly_AUC_genomics, test[,-1])
  perf_svm_poly_AUC_genomics <- confMat(pred_svm_poly_AUC_genomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_svm_poly_AUC_all = train(Histology~., data = all_AUC, method = "svmPoly", trControl = ctrl, tuneLength=5)
  pred_svm_poly_AUC_all <- predict(model_svm_poly_AUC_all, test[,-1])
  perf_svm_poly_AUC_all <- confMat(pred_svm_poly_AUC_all, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  
  ########## XGBoost
  model_xgboost_AUC_radiomics = train(Histology~., data = radiomics_data_AUC_Histology, method = "xgbTree", trControl = ctrl, tuneLength=5)
  pred_xgboost_AUC_radiomics <- predict(model_xgboost_AUC_radiomics, test[,-1])
  perf_xgboost_AUC_radiomics <- confMat(pred_xgboost_AUC_radiomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_xgboost_AUC_genomics = train(Histology~., data = genetic_data_AUC_histology, method = "xgbTree", trControl = ctrl, tuneLength=5)
  pred_xgboost_AUC_genomics <- predict(model_xgboost_AUC_genomics, test[,-1])
  perf_xgboost_AUC_genomics <- confMat(pred_xgboost_AUC_genomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_xgboost_AUC_all = train(Histology~., data = all_AUC, method = "xgbTree", trControl = ctrl, tuneLength=5)
  pred_xgboost_AUC_all <- predict(model_xgboost_AUC_all, test[,-1])
  perf_xgboost_AUC_all <- confMat(pred_xgboost_AUC_all, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  
  ####################################################################################
  ######################################## RFE #######################################
  ####################################################################################
  
  control_rfe = rfeControl(functions = rfFuncs, 
                           method = "repeatedcv", 
                           number = 5, repeats = 5) 
  
  radiomics <- train[,1:33]
  result_rfe_radiomics = rfe(x = radiomics[,-1], # 
                             y = as.factor(radiomics[,1]), 
                             sizes = c(5:(dim(radiomics)[2]*0.50)), rfeControl = control_rfe)
  radiomics_rfe <- cbind(as.factor(radiomics[,1]), subset(radiomics, select = result_rfe_radiomics$optVariables)) #[1:5]
  names(radiomics_rfe)[1] <- "Histology"
  
  
  genomics <- train[,c(1,34:5301)]
  result_rfe_genomics = rfe(x = genomics[,-1], 
                            y = genomics[,1], 
                            sizes = c(5, 50, 100, 250, 500, 1000), rfeControl = control_rfe)
  genomics_rfe <- cbind(as.factor(genomics[,1]), subset(genomics, select = result_rfe_genomics$optVariables)) #[1:5]
  names(genomics_rfe)[1] <- "Histology"
  
  all_rfe <- cbind(radiomics_rfe, genomics_rfe[,-1])
  
  
  radiomics_rfe$Histology <- plyr::revalue(radiomics_rfe$Histology, 
                                           c("1" = "Adenocarcinoma", 
                                             "0" = "Squamous_cell_carcinoma"))
  
  genomics_rfe$Histology <- plyr::revalue(genomics_rfe$Histology, 
                                          c("1" = "Adenocarcinoma", 
                                            "0" = "Squamous_cell_carcinoma"))
  
  all_rfe$Histology <- plyr::revalue(all_rfe$Histology, 
                                     c("1" = "Adenocarcinoma", 
                                       "0" = "Squamous_cell_carcinoma"))
  
  test$Histology <- plyr::revalue(test$Histology, 
                                  c("1" = "Adenocarcinoma", 
                                    "0" = "Squamous_cell_carcinoma"))
  
  #Cross-validation
  ctrl = trainControl(method="cv", number=5, returnResamp = "final", summaryFunction = prSummary, classProbs = TRUE,
                      savePredictions = T, verbose=F)
  
  ########## Elastic net
  
  model_en_RFE_radiomics = train(Histology~., data = radiomics_rfe, method = "glmnet", trControl = ctrl, tuneLength=5)
  pred_en_RFE_radiomics <- predict(model_en_RFE_radiomics, test[,-1])
  perf_en_RFE_radiomics <- confMat(pred_en_RFE_radiomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_en_RFE_genomics = train(Histology~., data = genomics_rfe, method = "glmnet", trControl = ctrl, tuneLength=5)
  pred_en_RFE_genomics <- predict(model_en_RFE_genomics, test[,-1])
  perf_en_RFE_genomics <- confMat(pred_en_RFE_genomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_en_RFE_all = train(Histology~., data = all_rfe, method = "glmnet", trControl = ctrl, tuneLength=5)
  pred_en_RFE_all <- predict(model_en_RFE_all, test[,-1])
  perf_en_RFE_all <- confMat(pred_en_RFE_all, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  
  ########## Random Forest
  model_rf_RFE_radiomics = train(Histology~., data = radiomics_rfe, method = "rf", trControl = ctrl, tuneLength=5)
  pred_rf_RFE_radiomics <- predict(model_rf_RFE_radiomics, test[,-1])
  perf_rf_RFE_radiomics <- confMat(pred_rf_RFE_radiomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all

  model_rf_RFE_genomics = train(Histology~., data = genomics_rfe, method = "rf", trControl = ctrl, tuneLength=5)
  pred_rf_RFE_genomics <- predict(model_rf_RFE_genomics, test[,-1])
  perf_rf_RFE_genomics <- confMat(pred_rf_RFE_genomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_rf_RFE_all = train(Histology~., data = all_rfe, method = "rf", trControl = ctrl, tuneLength=5)
  pred_rf_RFE_all <- predict(model_rf_RFE_all, test[,-1])
  perf_rf_RFE_all <- confMat(pred_rf_RFE_all, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all

  
  ########## Support Vector Machines - Linear
  model_svm_linear_RFE_radiomics = train(Histology~., data = radiomics_rfe, method = "svmLinear", trControl = ctrl, tuneLength=5)
  pred_svm_linear_RFE_radiomics <- predict(model_svm_linear_RFE_radiomics, test[,-1])
  perf_svm_linear_RFE_radiomics <- confMat(pred_svm_linear_RFE_radiomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_svm_linear_RFE_genomics = train(Histology~., data = genomics_rfe, method = "svmLinear", trControl = ctrl, tuneLength=5)
  pred_svm_linear_RFE_genomics <- predict(model_svm_linear_RFE_genomics, test[,-1])
  perf_svm_linear_RFE_genomics <- confMat(pred_svm_linear_RFE_genomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_svm_linear_RFE_all = train(Histology~., data = all_rfe, method = "svmLinear", trControl = ctrl, tuneLength=5)
  pred_svm_linear_RFE_all <- predict(model_svm_linear_RFE_all, test[,-1])
  perf_svm_linear_RFE_all <- confMat(pred_svm_linear_RFE_all, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all

  
  ########## Support Vector Machines - Radial
  model_svm_radial_RFE_radiomics = train(Histology~., data = radiomics_rfe, method = "svmRadial", trControl = ctrl, tuneLength=5)
  pred_svm_radial_RFE_radiomics <- predict(model_svm_radial_RFE_radiomics, test[,-1])
  perf_svm_radial_RFE_radiomics <- confMat(pred_svm_radial_RFE_radiomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_svm_radial_RFE_genomics = train(Histology~., data = genomics_rfe, method = "svmRadial", trControl = ctrl, tuneLength=5)
  pred_svm_radial_RFE_genomics <- predict(model_svm_radial_RFE_genomics, test[,-1])
  perf_svm_radial_RFE_genomics <- confMat(pred_svm_radial_RFE_genomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_svm_radial_RFE_all = train(Histology~., data = all_rfe, method = "svmRadial", trControl = ctrl, tuneLength=5)
  pred_svm_radial_RFE_all <- predict(model_svm_radial_RFE_all, test[,-1])
  perf_svm_radial_RFE_all <- confMat(pred_svm_radial_RFE_all, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  
  ########## Support Vector Machines - Polynomial
  model_svm_poly_RFE_radiomics = train(Histology~., data = radiomics_rfe, method = "svmPoly", trControl = ctrl, tuneLength=5)
  pred_svm_poly_RFE_radiomics <- predict(model_svm_poly_RFE_radiomics, test[,-1])
  perf_svm_poly_RFE_radiomics <- confMat(pred_svm_poly_RFE_radiomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_svm_poly_RFE_genomics = train(Histology~., data = genomics_rfe, method = "svmPoly", trControl = ctrl, tuneLength=5)
  pred_svm_poly_RFE_genomics <- predict(model_svm_poly_RFE_genomics, test[,-1])
  perf_svm_poly_RFE_genomics <- confMat(pred_svm_poly_RFE_genomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_svm_poly_RFE_all = train(Histology~., data = all_rfe, method = "svmPoly", trControl = ctrl, tuneLength=5)
  pred_svm_poly_RFE_all <- predict(model_svm_poly_RFE_all, test[,-1])
  perf_svm_poly_RFE_all <- confMat(pred_svm_poly_RFE_all, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  
  ########## XGBoost
  model_xgboost_RFE_radiomics = train(Histology~., data = radiomics_rfe, method = "xgbTree", trControl = ctrl, tuneLength=5)
  pred_xgboost_RFE_radiomics <- predict(model_xgboost_RFE_radiomics, test[,-1])
  perf_xgboost_RFE_radiomics <- confMat(pred_xgboost_RFE_radiomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_xgboost_RFE_genomics = train(Histology~., data = genomics_rfe, method = "xgbTree", trControl = ctrl, tuneLength=5)
  pred_xgboost_RFE_genomics <- predict(model_xgboost_RFE_genomics, test[,-1])
  perf_xgboost_RFE_genomics <- confMat(pred_xgboost_RFE_genomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_xgboost_RFE_all = train(Histology~., data = all_rfe, method = "xgbTree", trControl = ctrl, tuneLength=5)
  pred_xgboost_RFE_all <- predict(model_xgboost_RFE_all, test[,-1])
  perf_xgboost_RFE_all <- confMat(pred_xgboost_RFE_all, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all

  
  ####################################################################################
  ############################# Without Feature Selection ############################
  ####################################################################################
  
  radiomics <- train[,1:33]
  genomics <- train[,c(1,34:5301)]
  all <- cbind(radiomics, genomics[,-1])
  
  radiomics$Histology <- plyr::revalue(radiomics$Histology, 
                                       c("1" = "Adenocarcinoma", 
                                         "0" = "Squamous_cell_carcinoma"))
  
  genomics$Histology <- plyr::revalue(genomics$Histology, 
                                      c("1" = "Adenocarcinoma", 
                                        "0" = "Squamous_cell_carcinoma"))
  
  all$Histology <- plyr::revalue(all$Histology, 
                                 c("1" = "Adenocarcinoma", 
                                   "0" = "Squamous_cell_carcinoma"))
  
  ########## Elastic net
  
  model_en_radiomics = train(Histology~., data = radiomics, method = "glmnet", trControl = ctrl, tuneLength=5)
  pred_en_radiomics <- predict(model_en_radiomics, test[,-1])
  perf_en_radiomics <- confMat(pred_en_radiomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_en_genomics = train(Histology~., data = genomics, method = "glmnet", trControl = ctrl, tuneLength=5)
  pred_en_genomics <- predict(model_en_genomics, test[,-1])
  perf_en_genomics <- confMat(pred_en_genomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_en_all = train(Histology~., data = all, method = "glmnet", trControl = ctrl, tuneLength=5)
  pred_en_all <- predict(model_en_all, test[,-1])
  perf_en_all <- confMat(pred_en_all, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  
  ########## Random Forest
  model_rf_radiomics = train(Histology~., data = radiomics, method = "rf", trControl = ctrl, tuneLength=5)
  pred_rf_radiomics <- predict(model_rf_radiomics, test[,-1])
  perf_rf_radiomics <- confMat(pred_rf_radiomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_rf_genomics = train(Histology~., data = genomics, method = "rf", trControl = ctrl, tuneLength=5)
  pred_rf_genomics <- predict(model_rf_genomics, test[,-1])
  perf_rf_genomics <- confMat(pred_rf_genomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_rf_all = train(Histology~., data = all, method = "rf", trControl = ctrl, tuneLength=5)
  pred_rf_all <- predict(model_rf_all, test[,-1])
  perf_rf_all <- confMat(pred_rf_all, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all

  
  ########## Support Vector Machines - Linear
  model_svm_linear_radiomics = train(Histology~., data = radiomics, method = "svmLinear", trControl = ctrl, tuneLength=5)
  pred_svm_linear_radiomics <- predict(model_svm_linear_radiomics, test[,-1])
  perf_svm_linear_radiomics <- confMat(pred_svm_linear_radiomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_svm_linear_genomics = train(Histology~., data = genomics, method = "svmLinear", trControl = ctrl, tuneLength=5)
  pred_svm_linear_genomics <- predict(model_svm_linear_genomics, test[,-1])
  perf_svm_linear_genomics <- confMat(pred_svm_linear_genomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_svm_linear_all = train(Histology~., data = all, method = "svmLinear", trControl = ctrl, tuneLength=5)
  pred_svm_linear_all <- predict(model_svm_linear_all, test[,-1])
  perf_svm_linear_all <- confMat(pred_svm_linear_all, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
 
  
  ########## Support Vector Machines - Radial
  model_svm_radial_radiomics = train(Histology~., data = radiomics, method = "svmRadial", trControl = ctrl, tuneLength=5)
  pred_svm_radial_radiomics <- predict(model_svm_radial_radiomics, test[,-1])
  perf_svm_radial_radiomics <- confMat(pred_svm_radial_radiomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_svm_radial_genomics = train(Histology~., data = genomics, method = "svmRadial", trControl = ctrl, tuneLength=5)
  pred_svm_radial_genomics <- predict(model_svm_radial_genomics, test[,-1])
  perf_svm_radial_genomics <- confMat(pred_svm_radial_genomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_svm_radial_all = train(Histology~., data = all, method = "svmRadial", trControl = ctrl, tuneLength=5)
  pred_svm_radial_all <- predict(model_svm_radial_all, test[,-1])
  perf_svm_radial_all <- confMat(pred_svm_radial_all, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  
  ########## Support Vector Machines - Polynomial
  model_svm_poly_radiomics = train(Histology~., data = radiomics, method = "svmPoly", trControl = ctrl, tuneLength=5)
  pred_svm_poly_radiomics <- predict(model_svm_poly_radiomics, test[,-1])
  perf_svm_poly_radiomics <- confMat(pred_svm_poly_radiomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_svm_poly_genomics = train(Histology~., data = genomics, method = "svmPoly", trControl = ctrl, tuneLength=5)
  pred_svm_poly_genomics <- predict(model_svm_poly_genomics, test[,-1])
  perf_svm_poly_genomics <- confMat(pred_svm_poly_genomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_svm_poly_all = train(Histology~., data = all, method = "svmPoly", trControl = ctrl, tuneLength=5)
  pred_svm_poly_all <- predict(model_svm_poly_all, test[,-1])
  perf_svm_poly_all <- confMat(pred_svm_poly_all, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  
  ########## XGBoost
  model_xgboost_radiomics = train(Histology~., data = radiomics, method = "xgbTree", trControl = ctrl, tuneLength=5)
  pred_xgboost_radiomics <- predict(model_xgboost_radiomics, test[,-1])
  perf_xgboost_radiomics <- confMat(pred_xgboost_radiomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_xgboost_genomics = train(Histology~., data = genomics, method = "xgbTree", trControl = ctrl, tuneLength=5)
  pred_xgboost_genomics <- predict(model_xgboost_genomics, test[,-1])
  perf_xgboost_genomics <- confMat(pred_xgboost_genomics, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  model_xgboost_all = train(Histology~., data = all, method = "xgbTree", trControl = ctrl, tuneLength=5)
  pred_xgboost_all <- predict(model_xgboost_all, test[,-1])
  perf_xgboost_all <- confMat(pred_xgboost_all, test$Histology, verbose = FALSE, positive = "Adenocarcinoma")$all
  
  results <- cbind(perf_en_AUC_radiomics,  perf_en_AUC_genomics,  perf_en_AUC_all,  
                   perf_rf_AUC_radiomics,  perf_rf_AUC_genomics,  perf_rf_AUC_all,  
                   perf_svm_linear_AUC_radiomics,  perf_svm_linear_AUC_genomics,  perf_svm_linear_AUC_all,  
                   perf_svm_radial_AUC_radiomics,  perf_svm_radial_AUC_genomics,  perf_svm_radial_AUC_all,  
                   perf_svm_poly_AUC_radiomics,  perf_svm_poly_AUC_genomics,  perf_svm_poly_AUC_all,   
                   perf_xgboost_AUC_radiomics,  perf_xgboost_AUC_genomics,  perf_xgboost_AUC_all, 
                   perf_en_RFE_radiomics,  perf_en_RFE_genomics,  perf_en_RFE_all,  
                   perf_rf_RFE_radiomics,  perf_rf_RFE_genomics,  perf_rf_RFE_all,  
                   perf_svm_linear_RFE_radiomics,  perf_svm_linear_RFE_genomics,  perf_svm_linear_RFE_all,  
                   perf_svm_radial_RFE_radiomics,  perf_svm_radial_RFE_genomics,  perf_svm_radial_RFE_all,  
                   perf_svm_poly_RFE_radiomics,  perf_svm_poly_RFE_genomics,  perf_svm_poly_RFE_all,   
                   perf_xgboost_RFE_radiomics,  perf_xgboost_RFE_genomics,  perf_xgboost_RFE_all, 
                   perf_en_radiomics,  perf_en_genomics,  perf_en_all,  
                   perf_rf_radiomics,  perf_rf_genomics,  perf_rf_all,  
                   perf_svm_linear_radiomics,  perf_svm_linear_genomics,  perf_svm_linear_all,  
                   perf_svm_radial_radiomics,  perf_svm_radial_genomics,  perf_svm_radial_all,  
                   perf_svm_poly_radiomics,  perf_svm_poly_genomics,  perf_svm_poly_all,   
                   perf_xgboost_radiomics,  perf_xgboost_genomics,  perf_xgboost_all)

results <- as.data.frame(results)
colnames(results) <- c("EN_radiomics_AUC", "EN_genomics_AUC", "EN_all_AUC", 
                       "RF_radiomics_AUC", "RF_genomics_AUC", "RF_all_AUC", 
                       "SVM_linear_radiomics_AUC", "SVM_linear_genomics_AUC", "SVM_linear_all_AUC", 
                       "SVM_radial_radiomics_AUC", "SVM_radial_genomics_AUC", "SVM_radial_all_AUC",  
                       "SVM_poly_radiomics_AUC", "SVM_poly_genomics_AUC",  "SVM_poly_all_AUC", 
                       "Xgboost_radiomics_AUC", "Xgboost_genomics_AUC", "Xgboost_all_AUC",
                       "EN_radiomics_RFE", "EN_genomics_RFE", "EN_all_RFE", 
                       "RF_radiomics_RFE", "RF_genomics_RFE", "RF_all_RFE", 
                       "SVM_linear_radiomics_RFE", "SVM_linear_genomics_RFE", "SVM_linear_all_RFE", 
                       "SVM_radial_radiomics_RFE", "SVM_radial_genomics_RFE", "SVM_radial_all_RFE",  
                       "SVM_poly_radiomics_RFE", "SVM_poly_genomics_RFE",  "SVM_poly_all_RFE", 
                       "Xgboost_radiomics_RFE", "Xgboost_genomics_RFE", "Xgboost_all_RFE",
                       "EN_radiomics", "EN_genomics", "EN_all", 
                       "RF_radiomics", "RF_genomics", "RF_all", 
                       "SVM_linear_radiomics", "SVM_linear_genomics", "SVM_linear_all", 
                       "SVM_radial_radiomics", "SVM_radial_genomics", "SVM_radial_all",  
                       "SVM_poly_radiomics", "SVM_poly_genomics",  "SVM_poly_all", 
                       "Xgboost_radiomics", "Xgboost_genomics", "Xgboost_all")


  fname1 <- paste("data", i, "csv", sep = ".")
  fname1 <- file.path("~/", fname1)
  write.csv(sim_data, fname1, row.names = FALSE)
  
  fname2 <- paste("results", i, "csv", sep = ".")
  fname2 <- file.path("~/", fname2)
  write.csv(results, fname2, row.names = FALSE)
  
}  

end <- Sys.time()
end-start


